<a href="https://colab.research.google.com/github/ammadamir1122/Markov-Chain-Python-Implementation/blob/main/Markov_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 2

We have a research lab, which is working on a spaceship. Making fuel for the spaceship’s reactor core is a tricky process because of the exotic matter involved. It starts as raw ore, then during processing, begins randomly changing between forms, eventually reaching a stable form. There may be multiple stable forms that a sample could ultimately reach, not all of which are useful as fuel.

Research lab has tasked you to help the scientists increase fuel creation efficiency by predicting the end state of a given ore sample. You have carefully studied the different structures that the ore can take and which transitions it undergoes. It appears that, while random, the probability of each structure transforming is fixed. That is, each time the ore is in 1 state, it has the same probabilities of entering the next state (which might be the same state). You have recorded the observed transitions in a matrix. The others in the lab have hypothesized more exotic forms that the ore can become, but you haven't seen all of them.

Task 2.1

Write a function solution(m) that takes an array of array of nonnegative ints representing how many times that state has gone to the next state and return an array of ints for each terminal state giving the exact probabilities of each terminal state, represented as the numerator for each state, then the denominator for all of them at the end and in simplest form. The matrix is at most 10 by 10. It is guaranteed that no matter which state the ore is in, there is a path from that state to a terminal state. That is, the processing will always eventually end in a stable state. The ore starts in state 0. The denominator will fit within a signed 32-bit integer during the calculation, as long as the fraction is simplified regularly.

For example, consider the matrix m:

[

    [0,1,0,0,0,1], # s0, the initial state, goes to s1 and s5 with equal probability

    [4,0,0,3,2,0], # s1 can become s0, s3, or s4, but with different probabilities

    [0,0,0,0,0,0], # s2 is terminal, and unreachable (never observed in practice)

    [0,0,0,0,0,0], # s3 is terminal

    [0,0,0,0,0,0], # s4 is terminal

    [0,0,0,0,0,0], # s5 is terminal

]

So, we can consider different paths to terminal states, such as:

s0 -> s1 -> s3

s0 -> s1 -> s0 -> s1 -> s0 -> s1 -> s4

s0 -> s1 -> s0 -> s5

Tracing the probabilities of each, we find that s2 has probability 0

s3 has probability 3/14

s4 has probability 1/7

s5 has probability 9/14

So, putting that together, and making a common denominator, gives an answer in the form of [s2.numerator, s3.numerator, s4.numerator, s5.numerator, denominator] which is [0, 3, 2, 9, 14].

Test cases

Input: solution([[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0,0], [0, 0, 0, 0, 0]])

Output: [7, 6, 8, 21]

Input: solution([[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])

Output: [0, 3, 2, 9, 14]

Task 2.2

Create an API gateway using flask for the above solution. The API should be able to respond to the results after getting the input from the POST request.

In [53]:
from fractions import Fraction
import numpy as np
# Rearranging the matrix to markov's standard form
def markov_standard_form(m):
      l = len(m)
      for i in range(l):
        row_sum = sum(m[i])
        if row_sum == 0:
            m[i][i] = 1
        else:
            for j in range(l):
                m[i][j] = Fraction(m[i][j], row_sum)

In [54]:
def extract_identity_matrix(size, matrix):
    matrix_I = []
    for i in range(size):
        row = []
        for j in range(size):
            if i == j:
                row.append(1.0)
            else:
                row.append(0.0)
        matrix_I.append(row)
    return matrix_I


In [55]:
def submatrix(m, rows, cols):
    new_matrix = []
    for row in rows:
        current_row = []
        for col in cols:
            current_row.append(m[row][col])
        new_matrix.append(current_row)
    return new_matrix

In [56]:
def matrix_Q(matrix, transient_states):
    return submatrix(matrix, transient_states, transient_states)

In [57]:
def matrix_R(m, transient_states, absorbing_states):
    return submatrix(m, transient_states, absorbing_states)

**Since we are approaching towards limiting matrix and now we have Q, R and I matrices we will find FR where F = (I - Q)^-1**

In [58]:
# Lets subtract matrix I and matrix Q
def materices_subtraction(arr_I, arr_Q):
    """
    Subtract the elements of arr_Q from arr_I and return the resulting array of arrays
    """
    # Check that the arrays have the same shape
    if len(arr_I) != len(arr_Q) or len(arr_I[0]) != len(arr_Q[0]):
        raise ValueError("Arrays must have the same shape")
    
    # Create a new array of arrays to hold the result
    subtraction_result = []
    
    # Subtract the elements of the arrays element-wise
    for i in range(len(arr_I)):
        row = []
        for j in range(len(arr_I[i])):
            row.append(arr_I[i][j] - arr_Q[i][j])
        subtraction_result.append(row)
    return subtraction_result

**Now we have to find the inverse of the matrix called subtraction_result, which will be equal to F**

In [59]:
def materices_inverse(subtraction_result):
    try:
        matrix_F = np.linalg.inv(subtraction_result)
        return matrix_F
    except np.linalg.LinAlgError:
        # Matrix is singular (not invertible)
        return None

**At this point we already have calculated both the matrices matrix_F and matrix_R, now to get the final probabilities we have to multiply both the matrices and this will be our final function named as "solution(m)":**

In [60]:
def materices_multiplication(matrix_F, matrix_R):
    # Check if the matrices can be multiplied
    if len(matrix_F[0]) != len(matrix_R):
        return None
    else:
        # Initialize result matrix with zeros
        final_probabilities = [[0] * len(matrix_R[0]) for i in range(len(matrix_F))]
        # Multiply the matrices
        for i in range(len(matrix_F)):
            for j in range(len(matrix_F[0])):
                for k in range(len(matrix_F)):
                    final_probabilities[i][j] += matrix_F[i][k] * matrix_R[k][j]
        return final_probabilities

In [67]:
from flask import Flask
from flask import request

app = Flask(__name__)

@app.route('/get_final_probabilities', methods=['GET'])
def solution(m):

    transient_states = []
    absorbing_states = []
    for i in range(len(m)):
        row = m[i]
        if sum(row) == 0:
            absorbing_states.append(i)
        else:
            transient_states.append(i)

    markov_standard_form(m)

    Q = matrix_Q(m, transient_states)
    R = matrix_R(m, transient_states, absorbing_states)
    I = extract_identity_matrix(2, m)
    diff = materices_subtraction(I, matrix_Q)
    inverse = materices_inverse(diff)
    result = materices_multiplication(inverse, R)

if __name__ == '__main__':
    app.run(debug=True)